# Model building
* fully connected layer를 4개 이어붙여서 만든다.

In [4]:
# machine learning module
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

# ANN module
import torch
from torch import nn, optim                           # torch 에서 제공하는 신경망 기술, 손실함수, 최적화를 할 수 있는 함수들을 불러온다.
from torch.utils.data import DataLoader, Dataset      # 데이터를 모델에 사용할 수 있게 정리해주는 라이브러리.
import torch.nn.functional as F                       # torch 내의 세부적인 기능을 불러옴.

# Loss
from sklearn.metrics import mean_squared_error        # regression 문제의 모델 성능 측정을 위해서 MSE를 불러온다.

import matplotlib.pyplot as plt

In [5]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm

In [52]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.cuda.get_device_name(0))

cuda:0
GeForce RTX 2080 SUPER


# Data Loading

In [32]:
# train_X load
with open("../data/train_x_pandas.pickle", "rb") as f:
    train_x = pickle.load(f)

# train_Y load
with open("../data/train_y_pandas.pickle", "rb") as f:
    train_y = pickle.load(f)
    
# eval_X load
with open("../data/eval_x_pandas.pickle", "rb") as f:
    eval_x = pickle.load(f)
    
# eval_Y load
with open("../data/eval_y_pandas.pickle", "rb") as f:
    eval_y = pickle.load(f)

In [33]:
train_x.head(5)

,education,religion,bodyType,diet,age,smokes,drinks,pets
"(8195, 8195)",1.0,1.0,0.869592,1.0,1.00,1.0,1.0,2
"(8195, 8202)",0.5,0.5,0.934448,1.0,0.00,0.0,0.0,2
"(8195, 13)",0.5,0.5,0.787254,1.0,0.25,1.0,1.0,0
"(8195, 8207)",1.0,0.5,0.869592,1.0,0.25,1.0,1.0,0
"(8195, 17)",1.0,0.5,0.869592,1.0,0.50,1.0,1.0,2


In [34]:
train_y.head(5)

,cos_sim
"(8195, 8195)",10.000001
"(8195, 8202)",3.258006
"(8195, 13)",2.172685
"(8195, 8207)",3.288786
"(8195, 17)",4.524014


In [35]:
eval_x.head(5)

,education,religion,bodyType,diet,age,smokes,drinks,pets
"(27137, 27137)",1.0,1.0,1.000000,1.0,1.0,1.0,1.0,2
"(27137, 24734)",1.0,0.5,0.850984,1.0,0.0,1.0,1.0,0
"(27137, 19554)",0.5,0.5,1.000000,1.0,0.0,1.0,1.0,0
"(27137, 18219)",0.5,0.5,0.850984,1.0,0.0,1.0,1.0,0
"(27137, 18665)",0.5,0.1,0.850984,1.0,0.0,1.0,1.0,2


In [36]:
eval_y.head(5)

,cos
"(27137, 27137)",10.000000
"(27137, 24734)",5.448683
"(27137, 19554)",2.248607
"(27137, 18219)",0.838024
"(27137, 18665)",2.746272


# data scaling
* 데이터를 스케일링해서 표준화해준다.

In [37]:
# 데이터 스케일링
# sklearn에서 제공하는 MinMaxScaler 
# (X-min(X))/(max(X)-min(X))을 계산
scaler = MinMaxScaler() 
scaler.fit(train_x) 
train_x = scaler.transform(train_x)

scaler.fit(train_y)
train_y = scaler.transform(train_y)

In [38]:
 scaler.fit(eval_x) 
eval_x = scaler.transform(eval_x)

scaler.fit(eval_y)
eval_y = scaler.transform(eval_y)

# Dataset class

In [39]:
# torch의 Dataset 을 상속.
class TensorData(Dataset):

    def __init__(self, x_data, y_data):
        self.x_data = torch.FloatTensor(x_data)
        self.y_data = torch.FloatTensor(y_data)
        self.len = self.y_data.shape[0]

    def __getitem__(self, index):

        return self.x_data[index], self.y_data[index] 

    def __len__(self):
        return self.len

# Dataloader

In [40]:
# 학습 데이터, 시험 데이터 배치 형태로 구축하기
# batch size 32
trainsets = TensorData(train_x, train_y)
trainloader = torch.utils.data.DataLoader(trainsets, batch_size=32, shuffle=True)

testsets = TensorData(eval_x, eval_y)
testloader = torch.utils.data.DataLoader(testsets, batch_size=32, shuffle=False)

In [45]:
class Regressor(nn.Module):
    def __init__(self):
        super().__init__() # 모델 연산 정의
        self.fc1 = nn.Linear(8, 50, bias=True) # 입력층(13) -> 은닉층1(50)으로 가는 연산
        self.fc2 = nn.Linear(50, 100, bias=True) # 은닉층1(50) -> 은닉층2(30)으로 가는 연산
        self.fc3 = nn.Linear(100, 30, bias=True)
        self.fc4 = nn.Linear(30, 1, bias=True) # 은닉층2(30) -> 출력층(1)으로 가는 연산
        self.dropout = nn.Dropout(0.2) # 연산이 될 때마다 20%의 비율로 랜덤하게 노드를 없앤다.

    def forward(self, x): # 모델 연산의 순서를 정의
        x = F.relu(self.fc1(x)) # Linear 계산 후 활성화 함수 ReLU를 적용한다.  
        x = self.dropout(F.relu(self.fc2(x))) # 은닉층2에서 드랍아웃을 적용한다.(즉, 30개의 20%인 6개의 노드가 계산에서 제외된다.)
        x = self.dropout(F.relu(self.fc3(x)))
        x = F.relu(self.fc4(x)) # Linear 계산 후 활성화 함수 ReLU를 적용한다.  
      
        return x
    
# 주의 사항
# 드랍아웃은 과적합(overfitting)을 방지하기 위해 노드의 일부를 배제하고 계산하는 방식이기 때문에 절대로 출력층에 사용해서는 안 된다.

In [54]:
model = Regressor()
criterion = nn.MSELoss()

optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-7)

In [55]:
loss_ = [] # loss를 저장할 리스트.
n = len(trainloader)

val_rmse = []

model.to(device)

# 400 epoch training
for epoch in range(400):
    model.train()
    running_loss = 0.0 # 한 에폭이 돌 때 그안에서 배치마다 loss가 나온다. 즉 한번 학습할 때 그렇게 쪼개지면서 loss가 다 나오니 MSE를 구하기 위해서 사용한다.
    for i, data in tqdm(enumerate(trainloader, 0)): # 무작위로 섞인 32개의 데이터가 담긴 배치가 하나씩 들어온다.
    
        inputs, values = data # data에는 X, Y가 들어있다.
        inputs.to(device)
        values.to(device)
        optimizer.zero_grad() # 최적화 초기화.

        outputs = model(inputs) # 모델에 입력값을 넣어 예측값을 산출한다.
        loss = criterion(outputs, values) # 손실함수를 계산. error 계산.
        loss.backward() # 손실 함수를 기준으로 역전파를 설정한다.
        optimizer.step() # 역전파를 진행하고 가중치를 업데이트한다.

        running_loss += loss.item() # epoch 마다 평균 loss를 계산하기 위해 배치 loss를 더한다.
    loss_.append(running_loss/n) # MSE(Mean Squared Error) 계산
    
    print("training loss. epoch {%2d}: {%2.5f}".format(epoch, running_loss / n))
    
    # evaluation
    with torch.no_grad():
        model.eval()
        predictions = torch.tensor([], dtype=torch.float) # 예측값을 저장하는 텐서.
        actual = torch.tensor([], dtype=torch.float) # 실제값을 저장하는 텐서.
        
        for data in testloader:
            inputs, values = data
            inputs.to(device)
            values.to(device)
            outputs = model(inputs)
            
            predictions = torch.cat((predictions, outputs), 0) # dim 0
            actual = torch.cat((actual, values), 0) # dim 0
        
        # validation
        predictions = predictions.numpy() # 넘파이 배열로 변경.
        actual = actual.numpy() # 넘파이 배열로 변경.
        rmse = np.sqrt(mean_squared_error(predictions, actual)) # sklearn을 이용해 RMSE를 계산.
        val_rmse.append(rmse)
        
        print("epoch {%2d}. validation loss: {%3.5f}".format(epoch, rmse))


0it [00:00, ?it/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking arugment for argument mat1 in method wrapper_addmm)